# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 12 2022 10:24AM,244057,19,STR08122022-1,ACG North America LLC,Released
1,Aug 12 2022 10:17AM,244056,16,SSF-C4W33,Sartorius Stedim Filters Inc.,Released
2,Aug 12 2022 10:13AM,244045,10,PRF-35040,Bio-PRF,Released
3,Aug 12 2022 10:13AM,244045,10,PRF-35047,Bio-PRF,Released
4,Aug 12 2022 10:13AM,244045,10,PRF-35050,Bio-PRF,Released
5,Aug 12 2022 10:13AM,244045,10,PRF-35053,Bio-PRF,Released
6,Aug 12 2022 10:12AM,244044,10,8389552,Eywa Pharma Inc.,Released
7,Aug 12 2022 10:12AM,244044,10,8390686,Eywa Pharma Inc.,Released
8,Aug 12 2022 10:12AM,244044,10,8390700,Eywa Pharma Inc.,Released
9,Aug 12 2022 10:12AM,244044,10,8390701,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,244051,Released,1
39,244054,Executing,1
40,244054,Released,2
41,244056,Released,1
42,244057,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
244049,NaN,NaN,NaN,1.0
244051,NaN,NaN,NaN,1.0
244054,NaN,NaN,1.0,2.0
244056,NaN,NaN,NaN,1.0
244057,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
243961,0.0,10.0,9.0,0.0
243963,1.0,0.0,0.0,17.0
243968,0.0,0.0,0.0,1.0
243969,0.0,0.0,0.0,1.0
243971,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
243961,0.0,10,9,0
243963,1.0,0,0,17
243968,0.0,0,0,1
243969,0.0,0,0,1
243971,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,243961,0.0,10,9,0
1,243963,1.0,0,0,17
2,243968,0.0,0,0,1
3,243969,0.0,0,0,1
4,243971,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,243961,0.0,10,9,
1,243963,1.0,,,17
2,243968,0.0,,,1
3,243969,0.0,,,1
4,243971,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 12 2022 10:24AM,244057,19,ACG North America LLC
1,Aug 12 2022 10:17AM,244056,16,Sartorius Stedim Filters Inc.
2,Aug 12 2022 10:13AM,244045,10,Bio-PRF
6,Aug 12 2022 10:12AM,244044,10,Eywa Pharma Inc.
12,Aug 12 2022 9:58AM,244054,19,ACG North America LLC
15,Aug 12 2022 9:53AM,244051,16,Sartorius Stedim Filters Inc.
16,Aug 12 2022 9:40AM,244049,16,Sartorius Stedim Filters Inc.
17,Aug 12 2022 9:29AM,244043,20,"ACI Healthcare USA, Inc."
31,Aug 12 2022 9:25AM,244046,16,Sartorius Stedim Filters Inc.
32,Aug 12 2022 9:08AM,244037,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Aug 12 2022 10:24AM,244057,19,ACG North America LLC,0.0,,,1
1,Aug 12 2022 10:17AM,244056,16,Sartorius Stedim Filters Inc.,0.0,,,1
2,Aug 12 2022 10:13AM,244045,10,Bio-PRF,0.0,,,4
3,Aug 12 2022 10:12AM,244044,10,Eywa Pharma Inc.,0.0,,,6
4,Aug 12 2022 9:58AM,244054,19,ACG North America LLC,0.0,,1,2
5,Aug 12 2022 9:53AM,244051,16,Sartorius Stedim Filters Inc.,0.0,,,1
6,Aug 12 2022 9:40AM,244049,16,Sartorius Stedim Filters Inc.,0.0,,,1
7,Aug 12 2022 9:29AM,244043,20,"ACI Healthcare USA, Inc.",0.0,,,14
8,Aug 12 2022 9:25AM,244046,16,Sartorius Stedim Filters Inc.,0.0,,,1
9,Aug 12 2022 9:08AM,244037,15,"Virtus Pharmaceuticals, LLC",0.0,,5,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 10:24AM,244057,19,ACG North America LLC,1,,
1,Aug 12 2022 10:17AM,244056,16,Sartorius Stedim Filters Inc.,1,,
2,Aug 12 2022 10:13AM,244045,10,Bio-PRF,4,,
3,Aug 12 2022 10:12AM,244044,10,Eywa Pharma Inc.,6,,
4,Aug 12 2022 9:58AM,244054,19,ACG North America LLC,2,1,
5,Aug 12 2022 9:53AM,244051,16,Sartorius Stedim Filters Inc.,1,,
6,Aug 12 2022 9:40AM,244049,16,Sartorius Stedim Filters Inc.,1,,
7,Aug 12 2022 9:29AM,244043,20,"ACI Healthcare USA, Inc.",14,,
8,Aug 12 2022 9:25AM,244046,16,Sartorius Stedim Filters Inc.,1,,
9,Aug 12 2022 9:08AM,244037,15,"Virtus Pharmaceuticals, LLC",4,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 10:24AM,244057,19,ACG North America LLC,1,,
1,Aug 12 2022 10:17AM,244056,16,Sartorius Stedim Filters Inc.,1,,
2,Aug 12 2022 10:13AM,244045,10,Bio-PRF,4,,
3,Aug 12 2022 10:12AM,244044,10,Eywa Pharma Inc.,6,,
4,Aug 12 2022 9:58AM,244054,19,ACG North America LLC,2,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 10:24AM,244057,19,ACG North America LLC,1.0,NaN,NaN
1,Aug 12 2022 10:17AM,244056,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
2,Aug 12 2022 10:13AM,244045,10,Bio-PRF,4.0,NaN,NaN
3,Aug 12 2022 10:12AM,244044,10,Eywa Pharma Inc.,6.0,NaN,NaN
4,Aug 12 2022 9:58AM,244054,19,ACG North America LLC,2.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 10:24AM,244057,19,ACG North America LLC,1.0,0.0,0.0
1,Aug 12 2022 10:17AM,244056,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
2,Aug 12 2022 10:13AM,244045,10,Bio-PRF,4.0,0.0,0.0
3,Aug 12 2022 10:12AM,244044,10,Eywa Pharma Inc.,6.0,0.0,0.0
4,Aug 12 2022 9:58AM,244054,19,ACG North America LLC,2.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4636218,271.0,2.0,12.0
102,244014,1.0,0.0,0.0
15,732107,8.0,19.0,0.0
16,1220203,5.0,0.0,0.0
19,488111,3.0,1.0,0.0
20,975993,65.0,9.0,10.0
21,244017,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4636218,271.0,2.0,12.0
1,102,244014,1.0,0.0,0.0
2,15,732107,8.0,19.0,0.0
3,16,1220203,5.0,0.0,0.0
4,19,488111,3.0,1.0,0.0
5,20,975993,65.0,9.0,10.0
6,21,244017,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,271.0,2.0,12.0
1,102,1.0,0.0,0.0
2,15,8.0,19.0,0.0
3,16,5.0,0.0,0.0
4,19,3.0,1.0,0.0
5,20,65.0,9.0,10.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,271.0
1,102,Released,1.0
2,15,Released,8.0
3,16,Released,5.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,19,20,21
Status,,,,,,,
Completed,12.0,0.0,0.0,0.0,0.0,10.0,0.0
Executing,2.0,0.0,19.0,0.0,1.0,9.0,1.0
Released,271.0,1.0,8.0,5.0,3.0,65.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,19,20,21
0,Completed,12.0,0.0,0.0,0.0,0.0,10.0,0.0
1,Executing,2.0,0.0,19.0,0.0,1.0,9.0,1.0
2,Released,271.0,1.0,8.0,5.0,3.0,65.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,19,20,21
0,Completed,12.0,0.0,0.0,0.0,0.0,10.0,0.0
1,Executing,2.0,0.0,19.0,0.0,1.0,9.0,1.0
2,Released,271.0,1.0,8.0,5.0,3.0,65.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()